# Neural Machine Translation using a Transformer model

Finetune [T5](https://huggingface.co/t5-small) on the English-French subset of the [OPUS Books](https://huggingface.co/datasets/opus_books) dataset to translate English text to French.


## Install required libraries

In [ ]:
import re
import pandas as pd
from numpy import array, random, take
pd.options.mode.chained_assignment = None

In [ ]:

!pip install datasets
!pip install transformers
!pip install evaluate
!pip install sacrebleu
!pip install --upgrade --no-cache-dir gdown
# import locale
# locale.getpreferredencoding = lambda: "UTF-8"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 KB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 15.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 45.0 MB/s eta 0:00

## Imports

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import DataCollatorForSeq2Seq
import evaluate
from sklearn.model_selection import train_test_split
import numpy as np
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import pipeline
import pandas as pd
from datasets import load_dataset, load_metric, Dataset
import pandas as pd
import numpy as np
import string
import re
pd.options.mode.chained_assignment = None

## Download the dataset
Loading the English-French subset of the [OPUS Books](https://huggingface.co/datasets/opus_books) dataset:

In [ ]:
!gdown 1-nLLvtOF_92WxC0-Uo4wJ5gDeQ9moS_2

Downloading...
From: https://drive.google.com/uc?id=1-nLLvtOF_92WxC0-Uo4wJ5gDeQ9moS_2
To: /content/EN-FR.zip
100% 13.3M/13.3M [00:00<00:00, 164MB/s]


In [ ]:
!unzip /content/EN-FR.zip -d /content/data/

Archive:  /content/EN-FR.zip
  inflating: /content/data/dev.csv   
  inflating: /content/data/test.csv  
  inflating: /content/data/train.csv  


## Read Dataset

In [ ]:
df=pd.read_csv("/content/data/train.csv")

In [ ]:
for df_i in df:
    for _ in range(7): # Repeat the process of removing @Blabla, 7 time as some texts has "@Blabla" more than once
        for i in range(len(df_i)):
            for the_at_index in range(len(df_i['text'].iloc[i])):
                if df_i['text'].iloc[i][the_at_index] == '@':
                    for the_space_index in range(the_at_index,len(df_i['text'].iloc[i])):

                        if df_i['text'].iloc[i][the_space_index] == " ":
                            df_i['text'].iloc[i] = df_i['text'].iloc[i][0 : the_at_index : ] + df_i['text'].iloc[i][ the_space_index+1 : :]
                            break

                        elif(the_space_index == len(df_i['text'].iloc[i])-1):
                            df_i['text'].iloc[i] = df_i['text'].iloc[i][0 : the_at_index : ]
                            break

                    break

In [ ]:
for df_i in df:
    for _ in range(4): # Repeat the process of removing "http://t.co/Blabla", 9 time as some texts has "@Blabla" more than once
        for i in range(len(df_i)):
            if "http" in df_i['text'].iloc[i]:
                the_at_index = df_i['text'].iloc[i].find("http")
                for the_space_index in range(the_at_index,len(df_i['text'].iloc[i])):

                    if df_i['text'].iloc[i][the_space_index] == " ":

#
                        df_i['text'].iloc[i] = df_i['text'].iloc[i][0 : the_at_index : ] + df_i['text'].iloc[i][ the_space_index+1 : :]
#
                        break

                    elif(the_space_index == len(df_i['text'].iloc[i])-1):
                        the_space_index = len(df_i['text'].iloc[i])-1
#
                        df_i['text'].iloc[i] = df_i['text'].iloc[i][0 : the_at_index : ]
#

                        break

In [ ]:
def remove(text):
    return re.sub("\"|#|$|%|&|\'|\(|\)|\*|\+|,|-|\.|\/|:|;|<|=|>|[|\\|]|^|_|`|{||}~","",text)

def remove_non_ascii(text):
    ascii_chars = set(string.printable)
    return ''.join(filter(lambda x: x in ascii_chars, text))

In [ ]:
df['en'] = df['en'].apply(remove)
df['fr'] = df['fr'].apply(remove)

In [ ]:
df['en'] = df['en'].apply(remove_non_ascii)
df['fr'] = df['fr'].apply(remove_non_ascii)

In [ ]:
df.to_csv("/content/data/train.csv",index=False)

In [ ]:
dataset = load_dataset("csv", data_files="/content/data/train.csv")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-0c60c6e77d7d5abb/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['en', 'fr'],
        num_rows: 76251
    })
})

In [ ]:
dataset['train'][8]

{'en': 'The truth was that an idiotic ambition had alone impelled Camille to leave Vernon',
 'fr': 'La vrit tait quune ambition bte avait seule pouss Camille au dpart'}

In [ ]:
val_data = pd.read_csv('/content/data/dev.csv')
ds_val = Dataset.from_pandas(val_data)

test_data = pd.read_csv('/content/data/test.csv')
ds_test = Dataset.from_pandas(test_data)


dataset["validation"] = ds_val
dataset["test"] = ds_test

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['en', 'fr'],
        num_rows: 76251
    })
    validation: Dataset({
        features: ['en', 'fr'],
        num_rows: 25417
    })
    test: Dataset({
        features: ['en', 'fr'],
        num_rows: 25417
    })
})

### Load the ``T5`` tokenizer to process the English-French language pairs:

In [ ]:
model_name = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.9/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [ ]:
tokenizer(['At length, however, the remarks of her companions on her absence of mind aroused her, and she felt the necessity of appearing more like herself.'])

{'input_ids': [[486, 2475, 6, 983, 6, 8, 21029, 13, 160, 9663, 7, 30, 160, 8605, 13, 809, 1584, 32, 10064, 160, 6, 11, 255, 1800, 8, 16696, 13, 16069, 72, 114, 6257, 5, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

The preprocessing steps we need to create are:

1. Prefix the input with a prompt so T5 knows this is a translation task. Some models capable of multiple NLP tasks require prompting for specific tasks.

2. Tokenize the input (English) and target (French).


3. Truncate sequences to be no longer than the maximum length set by the `max_length` parameter.

In [ ]:
source_lang = "en"
target_lang = "fr"
prefix = "translate English to French: "

def preprocess_function(examples):
    inputs = [prefix + example for example in examples[source_lang]]
    targets = [example for example in examples[target_lang]]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True)
    return model_inputs

Apply the preprocessing function over the entire dataset:
- use the ``map`` method.
- ``batched=True`` to process multiple elements of the dataset at once.

In [ ]:
tokenized_data = dataset.map(preprocess_function, batched=True)

In [ ]:
print(tokenized_data['train'][10])

### Dynamically pad the inputs received

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model_name)

## Evaluation Metric
Including a metric for evaluating the model's performance. You can quickly load a evaluation method with the [Evaluate](https://huggingface.co/docs/evaluate/index) library. For this task, load the [SacreBLEU](https://huggingface.co/spaces/evaluate-metric/sacrebleu) metric.

In [ ]:
metric = evaluate.load("sacrebleu")

In [ ]:
# Model might produce some extra spaces or line breaks
# that are not present in the ground truth labels, and these could affect the evaluation metrics.
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]

    # decode the predicted values into human-readable text
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace all occurrences of -100 in the labels array with the ID of the pad token
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)

    # decode the labels values into human-readable text
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    # round it to four decimal places
    result = {k: round(v, 4) for k, v in result.items()}
    return result

# Training

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to("cuda")

At this point, only three steps remain:

1. Define your training hyperparameters in [Seq2SeqTrainingArguments](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Seq2SeqTrainingArguments).

2. Pass the training arguments to [Seq2SeqTrainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Seq2SeqTrainer) along with the model, dataset, tokenizer, data collator, and `compute_metrics` function.

3. Call [train()](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer.train) to finetune your model.

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    save_total_limit=3,
    num_train_epochs=2,
    # reduce memory usage
    fp16=True,

)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    data_collator=data_collator,
)

In [ ]:
trainer.train()

In [ ]:
 trainer.save_model('T5_checkpoint')

In [ ]:
!cp -r /content/T5_checkpoint /content/drive/MyDrive/

cp: cannot create directory '/content/drive/MyDrive/': No such file or directory


# Testing

In [ ]:
text = "translate English to French: Legumes share resources with nitrogen-fixing bacteria."
text2 = "my name is John"
text3 = 'he died'

In [ ]:
def predict(sentence):
  inputs = tokenizer(sentence, return_tensors="pt").input_ids
  model = AutoModelForSeq2SeqLM.from_pretrained("/content/T5_checkpoint")
  outputs = model.generate(inputs)
  outputs = tokenizer.decode(outputs[0], skip_special_tokens=True)
  return outputs

In [ ]:
pred = predict(text)
pred

'Les légumes partagent les ressources avec les bactéries fixatrice'

In [ ]:
pred = predict(text2)
pred

'Mein Name is John, John.'